In [3]:
import copernicusmarine as cm
import xarray as xr
import polars as pl
import numpy as np
from datetime import datetime

In [4]:
temp = pl.read_csv("../../data/temporal_subset.csv")
box = pl.read_csv("../../data/world_box.csv")

In [5]:
day = temp.filter(pl.col("mas_actual") == 0)["day"].to_list()[0]
month = temp.filter(pl.col("mas_actual") == 0)["month"].to_list()[0]
year = temp.filter(pl.col("mas_actual") == 0)["year"].to_list()[0]
min_time = f"{day}-{month}-{year}"
day = temp.filter(pl.col("mas_actual") == 1)["day"].to_list()[0]
month = temp.filter(pl.col("mas_actual") == 1)["month"].to_list()[0]
year = temp.filter(pl.col("mas_actual") == 1)["year"].to_list()[0]
max_time = f"{day}-{month}-{year}"
fecha_min = datetime.strptime(min_time, "%d-%m-%Y")
fecha_min_format = fecha_min.strftime("%Y-%m-%dT%H:%M:%S")
fecha_max = datetime.strptime(max_time, "%d-%m-%Y")
fecha_max_format = fecha_max.strftime("%Y-%m-%dT%H:%M:%S")
print("min", fecha_min_format)
print("max", fecha_max_format)

min 1999-06-15T00:00:00
max 2015-10-07T00:00:00


In [6]:
max_latitude = box["max_latitude"].to_list()[0]
min_latitude = box["min_latitude"].to_list()[0]
max_longitude = box["max_longitude"].to_list()[0]
min_longitude = box["min_longitude"].to_list()[0]
print("max", max_latitude, max_longitude)
print("min", min_latitude, min_longitude)

max -61.767468 179.988159
min -78.173 -179.998241


## Request

In [ ]:
def make_request_and_open_copernicus(dataset, output_name, start_time, end_time, max_latitude, min_latitude, max_longitude, min_longitude):
    cm.subset( 
        dataset_id = dataset,
        minimum_longitude = min_longitude,
        maximum_longitude = max_longitude,
        minimum_latitude = min_latitude,
        maximum_latitude = max_latitude,
        start_datetime = start_time,
        end_datetime = end_time,
        minimum_depth = 0,
        maximum_depth = 1,
        output_filename = output_name,
        output_directory = "copernicus-data"
    )
    nc_file = xr.open_dataset(f'data/copernicus-data/{output_name}')
    return nc_file

: 

In [ ]:
data = make_request_and_open_copernicus("cmems_mod_glo_phy_my_0.083deg_P1M-m", "Global_Ocean_Physics_Reanalysis.nc",
                                        fecha_min_format, fecha_max_format, max_latitude, min_latitude, max_longitude, min_longitude)
data

INFO - 2025-03-10T09:22:05Z - Downloading Copernicus Marine data requires a Copernicus Marine username and password, sign up for free at: https://data.marine.copernicus.eu/register


Copernicus Marine username:Copernicus Marine password:

INFO - 2025-03-10T09:22:16Z - Selected dataset version: "202311"
INFO - 2025-03-10T09:22:16Z - Selected dataset part: "default"
/home/pablo/miniconda3/envs/proy/lib/python3.9/site-packages/xarray/backends/plugins.py:80: RuntimeWarning: Engine 'earthkit' loading failed:

  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)
